In [1]:
from pydrake.all import (
    Simulator,
    StartMeshcat,
    DiagramBuilder,
)
from manipulation import FindResource, running_as_notebook

In [2]:
from manipulation.station import (
    AddPointClouds,
    AppendDirectives,
    LoadScenario,
    MakeHardwareStation,
)

In [3]:
import os
import sys
import torch
import logging
import numpy as np
from matplotlib import pyplot as plt

In [4]:
import sys
from utils import *

In [5]:
# Use AnyGrasp (TODO: add to args later)
use_anygrasp = False

In [19]:
### Start the visualizer ###
meshcat = StartMeshcat()
meshcat.AddButton("Stop meshcat")

### Diagram setup ###
builder = DiagramBuilder()
scenario = LoadScenario(
    filename="robots/spot_with_arm_and_floating_base_actuators_aligned_cameras.scenario.yaml"
)

### Add objects to scene ###
#scenario = AppendDirectives(scenario, filename="objects/added_object_directives.yaml")

station = builder.AddSystem(MakeHardwareStation(
    scenario=scenario,
    meshcat=meshcat,
    parser_preload_callback=lambda parser: parser.package_map().PopulateFromFolder(os.getcwd())
))

### Get subsystems ###
scene_graph = station.GetSubsystemByName("scene_graph")
plant = station.GetSubsystemByName("plant")


spot_plant = station.GetSubsystemByName(
"spot.controller" 
).get_multibody_plant_for_control()

from controller.spot_controller import SpotController
spot_controller = builder.AddSystem(
    SpotController(
        spot_plant,
        meshcat=meshcat,
        use_teleop=False,
    )
)

from controller.spot_arm_ik_controller import SpotArmIKController
spot_arm_ik_controller = builder.AddSystem(
    SpotArmIKController(
        spot_plant,
        use_teleop=False,
    )
)

INFO:drake:Meshcat listening for connections at https://02e87194-a9d3-4cfd-8d13-c9e760bd8905.deepnoteproject.com/7006/


In [7]:
from pydrake.all import LeafSystem, Context, AbstractValue, RigidTransform
class Controller_test_joint_position(LeafSystem):
    def __init__(self, station, builder, time_step=0.5, meshcat=None, visualize=False):
        super().__init__()
        self.meshcat = meshcat
        self.visualize = visualize

        self.target_arm_postion = self.DeclareDiscreteState()

        self.DeclareVectorOutputPort("target_arm_postion", 7, self.target_arm_postion)

    def set_arm_position(self, new_val):
        self.target_arm_postion = new_val

    def get_arm_position_output_port(self):
        return self.GetOutputPort("target_arm_postion")

class Controller_test_pose(LeafSystem):
    def __init__(self, station, builder, time_step=0.5, meshcat=None, visualize=False):
        super().__init__()
        self.meshcat = meshcat
        self.visualize = visualize

        self.target_pose = self.DeclareAbstractState(AbstractValue.Make(RigidTransform()))

        self.DeclareAbstractOutputPort(
            "target_pose", AbstractValue.Make(RigidTransform()), self.target_pose
        )

    def set_target_pose(self, new_val):
        self.target_pose = new_val

    def get_target_pose_output_port(self):
        return self.GetOutputPort("target_pose")


In [21]:
print(spot_controller)
print(spot_arm_ik_controller)

In [17]:
# Add controller
testing_inputs = ["arm_joint_values", "pose", "final_solution"] 
testing_input = testing_inputs[0]
    
if testing_input == testing_inputs[0]:
    target_arm_postion = np.array([1,1,1,1,1,1,1])

    controller_test = builder.AddNamedSystem(
                "controller_test", Controller_test_joint_position(station, builder)
            )
            
    print("working until here")    
    ### Here my stuff stops workin because I don't know how to implement the set function/leaf syste for this purpose

    controller_test.set_arm_position(target_arm_postion)
    builder.Connect(
                controller_test.get_arm_position_output_port(),
                spot_controller.GetInputPort("desired_arm_position"),
            )
        
elif testing_input == testing_inputs[1]:
    builder.Connect(
        spot_arm_ik_controller.GetOutputPort("desired_spot_arm_position"),
        spot_controller.GetInputPort("desired_arm_position"),
    )

    # TODO connect pose output from grasper to spot_arm_ik_controller.GetInputPort("desired_pose") for now with testing

    from pydrake.all import (
        RigidTransform,
        RotationMatrix,
    )
    goal_rotation = RotationMatrix(
        [
            [1, 0, 0],
            [0, -1, 0],
            [0, 0, -1],
        ]
    )
    goal_position = np.array([1, 0, 1.4])
    goal_pose = RigidTransform(goal_rotation, goal_position)

    controller_test = builder.AddNamedSystem(
                "controller_test", Controller_test_pose(station, builder)
            )

    print("working until here")
    ### Here again my stuff falls apart because I don't know how to implement the set function/leaf syste for this purpose
    controller_test.set_target_pose(goal_pose)

    builder.Connect(
        controller_test.GetOutputPort("target_pose"), #Would have to change output of navigation slightly
        spot_arm_ik_controller.GetInputPort("desired_pose"),
    )

    builder.Connect(
        navigation.GetOutputPort("desired_base_position"), #Would have to change output of navigation slightly
        spot_arm_ik_controller.GetInputPort("base_position"),
    )
    builder.Connect(
        navigation.GetOutputPort("desired_base_position"), #Would have to change output of navigation slightly
        spot_controller.GetInputPort("desired_base_position"),
    )

else:
    print("Define Testing Case")


builder.Connect(
    spot_controller.get_output_port(),
    station.GetInputPort("spot.desired_state"),
)

TypeError: DeclareDiscreteState(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.systems.framework.LeafSystem, model_vector: pydrake.systems.framework.BasicVector) -> pydrake.systems.framework.DiscreteStateIndex
    2. (self: pydrake.systems.framework.LeafSystem, vector: numpy.ndarray[numpy.float64[m, 1]]) -> pydrake.systems.framework.DiscreteStateIndex
    3. (self: pydrake.systems.framework.LeafSystem, num_state_variables: int) -> pydrake.systems.framework.DiscreteStateIndex

Invoked with: <__main__.Controller_test_joint_position object at 0x7f93630b8760>

In [ ]:
### Build and visualize diagram ###
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.set_name("tidyspot_diagram")
export_diagram_as_svg(diagram, "diagram.svg")

### Simulation setup ###
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

### Get subsystem contexts ###
station_context = station.GetMyMutableContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)

### Set initial Spot state ###
x0 = station.GetOutputPort("spot.state_estimated").Eval(station_context)
# Don't FixValue here - this will override the controller's commands
# station.GetInputPort("spot.desired_state").FixValue(station_context, x0)

### Run simulation ###
simulator.set_target_realtime_rate(1)
simulator.set_publish_every_time_step(True)

meshcat.Flush()  # Wait for the large object meshes to get to meshcat.

# meshcat.StartRecording()
simulator.AdvanceTo(2.0)

################
### TESTZONE ###
################

# # Display all camera images
# object_detector.display_all_camera_images(object_detector_context)

# # Display single image
# color_image = object_detector.get_color_image("frontleft", object_detector_context).data # Get color image from frontleft camera
# plt.imshow(color_image)
# plt.show()

# # Test anygrasp on frontleft camera
# grasp_selector.test_anygrasp_frontleft_pcd(to_point_cloud, context)

# # Keep meshcat alive
# meshcat.PublishRecording()

while not meshcat.GetButtonClicks("Stop meshcat"):
    pass

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=02e87194-a9d3-4cfd-8d13-c9e760bd8905' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>